In [1]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from datasets import Dataset
import pandas as pd
import torch
from tqdm import tqdm

In [13]:
torch.cuda.is_available()

True

In [2]:
import json

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.getOrCreate()

In [7]:
# test_df = spark.read.parquet("data/df_test.parquet").toPandas()
test_df = spark.read.parquet("data/df_test.parquet.gzip").toPandas()

In [8]:
display(test_df)

,text,group
0,"""Even explanations will not help in trusting [...",Computer Science
1,3D Printing of Invariant Manifolds in Dynamica...,Mathematics
2,A 10.8mW Mixed-Signal Simulated Bifurcation Is...,Electrical Engineering and Systems Science
3,A Comparative and Measurement-Based Study on R...,Computer Science
4,A Novel Approach To Implementing Knowledge Dis...,Computer Science
...,...,...
2176,Web3DB: Web 3.0 RDBMS for Individual Data Owne...,Computer Science
2177,When Gaussian Meets Surfel: Ultra-fast High-fi...,Computer Science
2178,douka: A universal platform of data assimilati...,Physics
2179,ms-Mamba: Multi-scale Mamba for Time-Series Fo...,Computer Science


In [10]:
with open("models/id2label.json", "r") as f:
    id2label = {int(k): v for k, v in json.load(f).items()}

id2label

{0: 'Computer Science',
 1: 'Physics',
 2: 'Mathematics',
 3: 'Electrical Engineering and Systems Science',
 4: 'Statistics',
 5: 'Quantitative Biology',
 6: 'Economics',
 7: 'Quantitative Finance'}

In [11]:
candidate_labels = [id2label[i] for i in range(len(id2label))]
candidate_labels

['Computer Science',
 'Physics',
 'Mathematics',
 'Electrical Engineering and Systems Science',
 'Statistics',
 'Quantitative Biology',
 'Economics',
 'Quantitative Finance']

In [14]:
zero_shot = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device="cuda")

Device set to use cuda


In [15]:
zero_shot_preds = []
batch_size = 32  # Adjust based on memory
texts = test_df["text"].tolist()

for i in tqdm(range(0, len(texts), batch_size)):
    batch = texts[i:i+batch_size]
    res_batch = zero_shot(batch, candidate_labels)
    
    if isinstance(res_batch, dict):  # Happens if batch size = 1
        zero_shot_preds.append(res_batch["labels"][0])
    else:
        zero_shot_preds.extend([res["labels"][0] for res in res_batch])

100%|██████████████████████████████████████████████████████████████████████████████████| 69/69 [10:13<00:00,  8.89s/it]


In [ ]:
# help(pipeline)

In [ ]:
zero_shot_preds

In [17]:
test_df_w_preds = test_df
test_df_w_preds['pred'] = zero_shot_preds

In [18]:
display(test_df_w_preds)

,text,group,pred
0,"""Even explanations will not help in trusting [...",Computer Science,Electrical Engineering and Systems Science
1,3D Printing of Invariant Manifolds in Dynamica...,Mathematics,Mathematics
2,A 10.8mW Mixed-Signal Simulated Bifurcation Is...,Electrical Engineering and Systems Science,Computer Science
3,A Comparative and Measurement-Based Study on R...,Computer Science,Electrical Engineering and Systems Science
4,A Novel Approach To Implementing Knowledge Dis...,Computer Science,Computer Science
...,...,...,...
2176,Web3DB: Web 3.0 RDBMS for Individual Data Owne...,Computer Science,Computer Science
2177,When Gaussian Meets Surfel: Ultra-fast High-fi...,Computer Science,Computer Science
2178,douka: A universal platform of data assimilati...,Physics,Computer Science
2179,ms-Mamba: Multi-scale Mamba for Time-Series Fo...,Computer Science,Computer Science


In [20]:
test_df_w_preds.to_parquet("data/df_test_pred_zeroshot.parquet.gzip")